In [125]:
from koeda import EDA, RD, RI, SR, RS
import pandas as pd
from tqdm import tqdm
import random

In [ ]:
from KorEDA import eda

c:\Users\bestm\Desktop\Dev\korean-text-augmentation\KorEDA\eda.py:138: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  words = [word for word in words if word is not ""]


In [ ]:
df = pd.read_csv('./dataset/train.csv')

In [ ]:
augmenter = EDA(morpheme_analyzer="Okt")

In [ ]:
num_aug = 4

In [ ]:
augmented_data = {"id": [], "text": [], "label": []}

for i, row in tqdm(df.iterrows(), total=len(df)):
    id = row["id"]
    label = row['label']
    text = row['text']
    
    # To prevent dupcliates, first augment 30 times and then sample 4 of them
    augmented_text = augmenter(text, repetition=30)
    augmented_text = list(set(augmented_text))
    #random sample
    augmented_text = random.sample(augmented_text, num_aug)
    
    assert len(augmented_text) <= num_aug

    augmented_data["id"].extend([id] * (num_aug + 1))
    augmented_data["text"].extend([text] + augmented_text)
    augmented_data["label"].extend([label] * (num_aug + 1))

100%|██████████| 2000/2000 [06:47<00:00,  4.91it/s]


In [ ]:
len(augmented_data["text"])

10000

In [ ]:
augmented_df = pd.DataFrame(augmented_data)

In [ ]:
augmented_df['label'].value_counts()

1    5000
0    5000
Name: label, dtype: int64

In [ ]:
augmented_df.to_csv("./dataset/train_aug_eda.csv", index=False)
augmented_df.drop(columns=["id"]).to_csv("./dataset/train_aug_eda_noid.csv", index=False)